In [ ]:
#pip install pygbif

Note: you may need to restart the kernel to use updated packages.


# STEP 00: SITE AND SPECIES DETERMINATION
In this notebook, code will be used to download occurrences of a species in from GBIF to see where the species resides. This will also help determine which sites to focus on.

In [23]:
# load packages

# reproducible file paths
import os
from glob import glob
import pathlib

# gbif packages
# need to install pygbif
import pygbif.occurrences as occ
import pygbif.species as species
from getpass import getpass

# unzipping and handling gbif data
import zipfile
from zipfile import ZipFile
import time

# deal with spatial data
import geopandas as gpd
import xrspatial

# deal w/ other types of data
import earthpy as et
import numpy as np
import pandas as pd
import rioxarray as rxr
import rioxarray.merge as rxrm

# invalied geometries
from shapely.geometry import MultiPolygon, Polygon

# packages for visualizing
import holoviews as hv
import hvplot.pandas
import hvplot.xarray

In [3]:
# make reproducible file paths
data_dir = os.path.join(
    # home directory
    pathlib.Path.home(),

    # eda directory
    'earth-analytics',
    'data',

    # project directory
    'spring-2025-habitat-suitability'
)

# make the directory
os.makedirs(data_dir, exist_ok=True)

In [4]:
# set gbif dir
gbif_dir = os.path.join(data_dir, 'gbif_dir')

In [5]:
# access gbif
reset_credentials = True

# enter gbif username, password, and email
credentials = dict(
    GBIF_USER=(input, 'GBIF username'),
    GBIF_PWD=(getpass, 'GBIF password'),
    GBIF_EMAIL=(input, 'GBIF email')
)

for env_variable, (prompt_func, prompt_text) in credentials.items():
    # delete credential from the environment if requested
    if reset_credentials and (env_variable in os.environ):
        os.environ.pop(env_variable)
    
    # ask for credential and save to environment
    if not env_variable in os.environ:
        os.environ[env_variable] = prompt_func(prompt_text)

In [6]:
# species names
species_name = 'Sequoia sempervirens'

# species info for gbif
species_info = species.name_lookup(species_name,
                                   rank = 'SPECIES')

# grab first result
first_result = species_info['results'][0]

# get species key
species_key = first_result['nubKey']

# check first_result
first_result['species'], species_key

('Sequoia sempervirens', 2683909)

In [7]:
#save species code
species_key = 2683909

In [8]:
# set a file pattern 
gbif_pattern = os.path.join(gbif_dir,
                            '*csv')

# download it once
if not glob(gbif_pattern):
    # submit query to GBIF to get all data from all years
    gbif_query = occ.download([
        f'speciesKey = {species_key}',
        'hasCoordinate = True'
    ])

    # only download once
    if not 'GBIF_DOWNLOAD_KEY' in os.environ:
        os.environ['GBIF_DOWNLOAD_KEY'] = gbif_query[0]
        download_key = os.environ['GBIF_DOWNLOAD_KEY'] 

        # wait for downlaod to build
        wait = occ.download_meta(download_key)['status']
        while not wait == 'SUCCEEDED' :
            wait = occ.download_meta(download_key)['status']
            time.sleep(5)
    # download the data
    download_info = occ.download_get(
        os.environ['GBIF_DOWNLOAD_KEY'],
        path = data_dir
    )

    # unzip it
    with zipfile.ZipFile(download_info['path']) as download_zip:
        download_zip.extractall(path = gbif_dir)

# find csv file path
gbif_path = glob(gbif_pattern)[0]

INFO:Your download key is 0005703-250227182400228
INFO:Download file size: 1211229 bytes
INFO:On disk at C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability/0005703-250227182400228.zip


redwood gbif download citation:

GBIF.org (05 March 2025) GBIF Occurrence Download  https://doi.org/10.15468/dl.yh4ygk

In [9]:
# open gbif data
gbif_df = pd.read_csv(
    gbif_path,
    delimiter = '\t'
)

# check dataframe
gbif_df.head()

C:\Users\riede\AppData\Local\Temp\ipykernel_6160\704011509.py:2: DtypeWarning: Columns (10,46) have mixed types. Specify dtype option on import or set low_memory=False.
  gbif_df = pd.read_csv(


,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
0,986394934,2429287b-ef65-4cfd-afcc-11cc3ba95cca,urn:catalog:ALTA-VP:132704,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,"Deyholos, M. K.",NaN,CC0_1_0,University of Alberta Museums,"Deyholos, M. K.",NaN,NaN,2025-02-27T20:43:10.577Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84;OCCURRENCE_STATUS...
1,930742189,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-kaufmannm/...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,Maurice J. Kaufmann,1977-01-01T00:00:00,CC0_1_0,NaN,Maurice J. Kaufmann,NaN,native,2025-02-06T17:33:39.109Z,StillImage,NaN
2,930742172,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-kaufmannm/...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,Maurice J. Kaufmann,1977-01-01T00:00:00,CC0_1_0,NaN,Maurice J. Kaufmann,NaN,native,2025-02-06T17:33:43.284Z,StillImage,NaN
3,930742166,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-kaufmannm/...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,Maurice J. Kaufmann,1977-01-01T00:00:00,CC0_1_0,NaN,Maurice J. Kaufmann,NaN,native,2025-02-06T17:33:42.424Z,StillImage,NaN
4,930741653,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-baskauf/41...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,Steven J. Baskauf,2005-07-28T00:00:00,CC0_1_0,NaN,Steven J. Baskauf,NaN,native,2025-02-06T17:33:40.327Z,StillImage,NaN


In [10]:
# see all gbif_df columns
gbif_df.columns

Index(['gbifID', 'datasetKey', 'occurrenceID', 'kingdom', 'phylum', 'class',
       'order', 'family', 'genus', 'species', 'infraspecificEpithet',
       'taxonRank', 'scientificName', 'verbatimScientificName',
       'verbatimScientificNameAuthorship', 'countryCode', 'locality',
       'stateProvince', 'occurrenceStatus', 'individualCount',
       'publishingOrgKey', 'decimalLatitude', 'decimalLongitude',
       'coordinateUncertaintyInMeters', 'coordinatePrecision', 'elevation',
       'elevationAccuracy', 'depth', 'depthAccuracy', 'eventDate', 'day',
       'month', 'year', 'taxonKey', 'speciesKey', 'basisOfRecord',
       'institutionCode', 'collectionCode', 'catalogNumber', 'recordNumber',
       'identifiedBy', 'dateIdentified', 'license', 'rightsHolder',
       'recordedBy', 'typeStatus', 'establishmentMeans', 'lastInterpreted',
       'mediaType', 'issue'],
      dtype='object')

In [11]:
# make gbif_df spatial
gbif_gdf = (
    gpd.GeoDataFrame(
        gbif_df,
        geometry = gpd.points_from_xy(
            # x value comes from decimalLongitude
            gbif_df.decimalLongitude,
            # y value comes from decimalLatitude
            gbif_df.decimalLatitude
        ),
        # assign a crs to the gdf
        crs = 'EPSG:4326'
    )
)

# check gbif_gdf
gbif_gdf.head()

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue,geometry
0,986394934,2429287b-ef65-4cfd-afcc-11cc3ba95cca,urn:catalog:ALTA-VP:132704,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,NaN,CC0_1_0,University of Alberta Museums,"Deyholos, M. K.",NaN,NaN,2025-02-27T20:43:10.577Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84;OCCURRENCE_STATUS...,POINT (-113.51667 53.51667)
1,930742189,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-kaufmannm/...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,1977-01-01T00:00:00,CC0_1_0,NaN,Maurice J. Kaufmann,NaN,native,2025-02-06T17:33:39.109Z,StillImage,NaN,POINT (-124.0411 41.3232)
2,930742172,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-kaufmannm/...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,1977-01-01T00:00:00,CC0_1_0,NaN,Maurice J. Kaufmann,NaN,native,2025-02-06T17:33:43.284Z,StillImage,NaN,POINT (-124.0411 41.3232)
3,930742166,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-kaufmannm/...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,1977-01-01T00:00:00,CC0_1_0,NaN,Maurice J. Kaufmann,NaN,native,2025-02-06T17:33:42.424Z,StillImage,NaN,POINT (-124.0411 41.3232)
4,930741653,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-baskauf/41...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,2005-07-28T00:00:00,CC0_1_0,NaN,Steven J. Baskauf,NaN,native,2025-02-06T17:33:40.327Z,StillImage,NaN,POINT (-124.087 41.79557)


In [13]:
# plot gbif_gdf to see where the occurrences are
gbif_gdf.hvplot(
    # treat the plot as geographic and assume lat/lon coordinates
    geo = True,
    # overlay the plot on EsriImagery tiles
    tiles = 'EsriImagery',
    # set title
    title = 'Coast Redwood (Sequoia sempervirens) Occurrences From GBIF',
    # set fill and line color
    fill_color = None, line_color = 'limegreen',
    # set frame size
    frame_width = 700
)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]

Site determination - 
* Uinta national Forest
    * [Forest Resources of the Uinta National Forest](https://www.fs.usda.gov/rm/pubs_series/forest_resources/uinta.pdf)

* Kaibab National Forest
    * [About the Forest](https://www.fs.usda.gov/main/kaibab/about-forest)
    
The forest url is from here: https://data-usfs.hub.arcgis.com/datasets/usfs::fs-national-forests-dataset-us-forest-service-proclaimed-forests/about

In [41]:
# set up national forest url
forest_url = "https://apps.fs.usda.gov/arcx/rest/services/EDW/EDW_ProclaimedForestBoundaries_01/MapServer/0/query?outFields=*&where=1%3D1&f=geojson"

# set up path to save forest data
forest_dir = os.path.join(data_dir, 'forest-dir')
os.makedirs(forest_dir, exist_ok=True)


# Join forest shapefile path
forest_path = os.path.join(forest_dir, 'S_USA.ProclaimedForestBoundaries.shp')

# Only download once
if not os.path.exists(forest_path):
    forest_gdf = gpd.read_file(forest_url)
    forest_gdf.to_file(forest_path)

# Create forest_gdf
forest_gdf = gpd.read_file(forest_path)

In [42]:
# Check forest_gdf
forest_gdf

,OBJECTID,PROCLAIMED,FORESTNAME,GIS_ACRES,SHAPE.AREA,SHAPE.LEN,geometry
0,200571,66329010328,Bighorn National Forest,1.112646e+06,0.509899,4.943363,"POLYGON ((-107.54298 44.93779, -107.54298 44.9..."
1,200572,93007010328,Gifford Pinchot National Forest,1.532173e+06,0.722290,11.679336,"MULTIPOLYGON (((-122.27148 45.75119, -122.2711..."
2,200573,96812010328,Manti-La Sal National Forest,1.337654e+06,0.561732,9.953308,"MULTIPOLYGON (((-111.40257 39.98081, -111.4009..."
3,200574,96813010328,Uinta National Forest,9.617430e+05,0.411377,6.874785,"MULTIPOLYGON (((-111.55131 40.59378, -111.5513..."
4,200575,105935010328,Kaibab National Forest,1.601003e+06,0.647156,9.140002,"MULTIPOLYGON (((-112.06366 36.8781, -112.06366..."
5,200576,106640010328,Fremont National Forest,1.713917e+06,0.760518,11.282002,"MULTIPOLYGON (((-120.53326 42.57082, -120.5285..."
6,200577,106887010328,Mt. Baker National Forest,1.317677e+06,0.648567,10.659633,"MULTIPOLYGON (((-121.75954 48.99733, -121.7512..."
7,200578,107266010328,Olympic National Forest,6.958685e+05,0.337426,8.179740,"MULTIPOLYGON (((-123.23508 48.01368, -123.2297..."
8,200579,107474010328,Wallowa National Forest,1.064857e+06,0.496222,9.027734,"MULTIPOLYGON (((-117.56261 45.48274, -117.5625..."
9,200580,108201010328,Wenatchee National Forest,1.963052e+06,0.952602,15.707300,"MULTIPOLYGON (((-121.17501 47.19079, -121.1750..."


In [34]:
# Create individual gdf for Uinta
uinta_forest_gdf = forest_gdf[forest_gdf['FORESTNAME']=='Uinta National Forest']

# Plot Uinta National Forest
uinta_forest_gdf.hvplot(
    # treat the plot as geographic and assume lat/lon coordinates
    geo = True,
    # overlay the plot on EsriImagery tiles
    tiles = 'EsriImagery',
    # set title
    title = 'Uinta National Forest',
    # set fill and line color
    fill_color = None, line_color = 'pink',
    frame_width = 300
)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [61]:
# Create individual gdf for Kaibab
kaibab_forest_gdf = forest_gdf[forest_gdf['FORESTNAME']=='Kaibab National Forest']

# Plot Uinta National Forest
kaibab_forest_gdf.hvplot(
    # treat the plot as geographic and assume lat/lon coordinates
    geo = True,
    # overlay the plot on EsriImagery tiles
    tiles = 'EsriImagery',
    # set title
    title = 'Kaibab National Forest',
    # set fill and line color
    fill_color = None, line_color = 'blue',
    # set frame width
    frame_width = 300
)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [43]:
# intersect aspen occurrence with forest_gdf
# only keep the occurrences that are in the forest_gdf shapefile
gbif_forest_intersection = gpd.overlay(gbif_gdf, forest_gdf, how = 'intersection')

In [44]:
# how many occurrences per site?
value_counts = gbif_forest_intersection['FORESTNAME'].value_counts()
value_counts

FORESTNAME
Angeles National Forest    2
Name: count, dtype: int64

In [67]:
# combine both sites into one
sites_gdf = gpd.GeoDataFrame(pd.concat([uinta_forest_gdf, kaibab_forest_gdf], ignore_index=True))
sites_gdf

,OBJECTID,PROCLAIMED,FORESTNAME,GIS_ACRES,SHAPE.AREA,SHAPE.LEN,geometry
0,200574,96813010328,Uinta National Forest,961743.024,0.411377,6.874785,"MULTIPOLYGON (((-111.55131 40.59378, -111.5513..."
1,200575,105935010328,Kaibab National Forest,1601002.978,0.647156,9.140002,"MULTIPOLYGON (((-112.06366 36.8781, -112.06366..."


In [68]:
# Plot both National Forests
sites_gdf.hvplot(
    # treat the plot as geographic and assume lat/lon coordinates
    geo = True,
    # overlay the plot on EsriImagery tiles
    tiles = 'EsriImagery',
    # set title
    title = 'Kaibab & Uinta National Forests',
    # set fill and line color
    fill_color = None, line_color = 'white',
    # set frame width
    frame_width = 400
)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]